In [21]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

import graphviz 
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.metrics import top_k_accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.decomposition import PCA

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import f1_score

### Carregamento do conjunto de dados de todos os anos

In [22]:
enade_todos_anos = pd.read_csv("../teste_OHE_curso_treinamento.csv")

### Cross-validation (k = 5)

In [23]:
k = 5

In [24]:
array_folds = np.array_split(enade_todos_anos, k)

In [25]:
for i_fold in range(k):
    
    # Separação dos folds
    
    folds_treinamento = array_folds.copy()
    fold_teste = array_folds[i_fold]
    
    del folds_treinamento[i_fold]
    
    folds_treinamento = pd.concat(folds_treinamento, sort=False)
    
    # Separação em X e y
    
    numero_caracteristicas = enade_todos_anos.shape[1] - 1
    
    X_folds_treinamento = folds_treinamento.iloc[:, 0:numero_caracteristicas]
    y_folds_treinamento = folds_treinamento.iloc[:, -1]
    
    X_fold_teste = fold_teste.iloc[:, 0:numero_caracteristicas]
    y_fold_teste = fold_teste.iloc[:, -1]
    
    # Normalização de treino e teste
    
    normalizador_treinamento = StandardScaler()
    
    normalizador_treinamento.fit(X_folds_treinamento)
    treinamento_normalizado = normalizador_treinamento.transform(X_folds_treinamento)
    
    normalizador_teste = StandardScaler()
    
    normalizador_teste.fit(X_fold_teste)
    teste_normalizado = normalizador_teste.transform(X_fold_teste)
    
    # Busca por hyperparâmetros
    
    parametros_para_busca = {"C": [0.01, 0.1, 1, 10, 100], 
              "kernel": ["linear", "poly", "rbf"],
              "gamma": [0.1, 1, 10],
              "tol": [0.1, 0.001, 0.00001]
             }
    
    classificador = SVC(probability=True)
    
    busca = HalvingGridSearchCV(classificador, parametros_para_busca, scoring="f1_weighted").fit(treinamento_normalizado, y_folds_treinamento)
    
    busca.best_estimator_.fit(treinamento_normalizado, y_folds_treinamento)
    y_predito = busca.best_estimator_.predict(teste_normalizado)
    
    melhor_classificador = str(busca.best_estimator_)
    resultado = f1_score(y_fold_teste, y_predito, average='weighted')
    
    print(f"Teste: fold {i_fold}")
    print(f"Classificador: \n\n{melhor_classificador}\n")
    print(f"Resultado: {resultado}\n")

Teste: fold 0
Classificador: 

SVC(C=0.1, gamma=0.1, kernel='linear', probability=True, tol=1e-05)

Resultado: 0.5199365127119691

Teste: fold 1
Classificador: 

SVC(C=0.01, gamma=1, kernel='linear', probability=True, tol=0.1)

Resultado: 0.4782876615620517

Teste: fold 2
Classificador: 

SVC(C=10, gamma=10, kernel='linear', probability=True, tol=0.1)

Resultado: 0.4458703365480332

Teste: fold 3
Classificador: 

SVC(C=0.1, gamma=0.1, kernel='linear', probability=True)

Resultado: 0.4502946130400957

Teste: fold 4
Classificador: 

SVC(C=0.01, gamma=1, kernel='linear', probability=True)

Resultado: 0.44753042331424303



### Matriz de Confusão

In [ ]:
y_pred_HGS = search.predict(X_test)

In [ ]:
cm_HGS = confusion_matrix(y_test, y_pred_HGS, labels=search.classes_)
disp_HGS = ConfusionMatrixDisplay(confusion_matrix=cm_HGS,
                              display_labels=search.classes_)
disp_HGS.plot()
plt.show()